In [1]:
import gzip
import nltk
import pandas as pd
import numpy as np
import pickle

from collections import Counter
from nltk.corpus import stopwords
import utils as my_utils

In [2]:
min_words = 8
max_words = 70

In [3]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [24]:
df = getDF('datasets/reviews_Musical_Instruments_5.json.gz')

In [25]:
df.shape

(10261, 9)

In [26]:
df.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014"
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013"


# Cleaning

In [27]:
%%time
df['cleaned'] = my_utils.preprocess(df['reviewText'])

CPU times: user 6.46 s, sys: 24.5 ms, total: 6.49 s
Wall time: 6.49 s


In [28]:
df.head(2)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,cleaned
0,A2IBPI20UZIR0U,1384719342,"cassandra tu ""Yeah, well, that's just like, u...","[0, 0]","Not much to write about here, but it does exac...",5.0,good,1393545600,"02 28, 2014","[much, write, doe, exactly, supposed, filter, ..."
1,A14VAT5EAX3D9S,1384719342,Jake,"[13, 14]",The product does exactly as it should and is q...,5.0,Jake,1363392000,"03 16, 2013","[product, doe, exactly, quite, affordable, rea..."


In [29]:
Counter(df['overall'])

Counter({5.0: 6938, 3.0: 772, 4.0: 2084, 2.0: 250, 1.0: 217})

In [30]:
df['remove'] = df['cleaned'].apply(lambda x: True if len(x)<min_words and len(x)>max_words else False)

In [31]:
df[df['remove']==True]

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,cleaned,remove


In [32]:
for idx, i in enumerate(df['cleaned']):
    if len(i) < min_words and len(i) > max_words:
        print(idx)

In [33]:
df = df.drop(columns=['remove'])

In [34]:
df['text'] = df['cleaned'].apply(lambda x: " ".join(x))

In [35]:
count_matrix, _, _, _ = my_utils.processReviews(df['text'].values)

In [36]:
count_matrix.shape

(10261, 2000)

In [37]:
barren = np.where(count_matrix.sum(1)==0)[0]

In [38]:
barren

array([2927, 4401, 8739, 9175, 9306, 9313, 9342])

In [39]:
for i in barren:
    df.drop(i, inplace=True)

In [40]:
df = df.rename(columns={'overall': 'sentiment'})

In [41]:
df.to_pickle("datasets/datadf_amazon_musical", protocol=2)

# For Training Embeddings

In [ ]:
questions = []
answers = []

for idx, asin in enumerate(df['questions'].values):
    for question in asin:
        questions.append(question['questionText'])
        for answer in question['answers']:
            answers.append(answer['answerText'])

In [ ]:
train_data = questions + answers

In [ ]:
train_data = pd.DataFrame(train_data)

In [ ]:
train_data.head()

In [ ]:
%%time
train_data['cleaned'] = my_utils.preprocess(train_data[0])

In [ ]:
train_data.head()

In [ ]:
training_data = []
for i in train_data['cleaned'].values:
    if len(i) > 2:
        training_data.append(" ".join(i))

In [ ]:
training_data = pd.DataFrame(training_data)

In [ ]:
training_data.shape

In [ ]:
p = "\n".join(training_data[0].values)

In [ ]:
f = open("nongit_resources/train_amazon_QA_without_stopwords.txt", 'w')
f.write(p)
f.close()